In [11]:
import pandas as pd

In [12]:
d = {
    "Los Angeles Clippers": "LA Clippers"
}

d_columns = {
    "home_team": d,
    "away_team": d
}

In [13]:
df = pd.read_json('Odds.json')
df = df[['commence_time', 'home_team', 'away_team', 'bookmakers']]

In [14]:
def unpack_bookmakers(dict_apps_odds):
    final_dic = {}
    for app in dict_apps_odds:
        app_name = app['title']
        dic_team_odd = []
        for team_odd in app['markets'][0]['outcomes']:
            dic_team_odd.append(team_odd['price'])
        final_dic[app_name]= dic_team_odd
    return final_dic

In [15]:
def find_best_bookmaker(dict_apps_odds):
    min_avg = sum(dict_apps_odds[list(dict_apps_odds.keys())[0]])/2
    best_bookmaker = list(dict_apps_odds.keys())[0]
    for key in dict_apps_odds.keys():
        avg = sum(dict_apps_odds[key])/2
        if avg < min_avg:
            min_avg = avg
            best_bookmaker = key
    return best_bookmaker, dict_apps_odds[key][0], dict_apps_odds[key][1]

df.bookmakers = df.bookmakers.apply(lambda x: unpack_bookmakers(x))    
df['bookmaker'], df['odd_home'], df['odd_away'] = zip(*df.bookmakers.apply(find_best_bookmaker))
df = df.drop(columns = 'bookmakers').replace(d_columns)
df

,commence_time,home_team,away_team,bookmaker,odd_home,odd_away
0,2022-12-01 00:10:00+00:00,Orlando Magic,Atlanta Hawks,WynnBET,1.44,2.63
1,2022-12-01 00:10:00+00:00,Cleveland Cavaliers,Philadelphia 76ers,WynnBET,1.55,2.38
2,2022-12-01 00:40:00+00:00,Boston Celtics,Miami Heat,WynnBET,1.25,4.00
3,2022-12-01 00:40:00+00:00,Brooklyn Nets,Washington Wizards,WynnBET,1.40,2.80
4,2022-12-01 00:40:00+00:00,New York Knicks,Milwaukee Bucks,WynnBET,1.44,2.63
5,2022-12-01 01:10:00+00:00,Minnesota Timberwolves,Memphis Grizzlies,WynnBET,1.57,2.30
6,2022-12-01 01:10:00+00:00,New Orleans Pelicans,Toronto Raptors,FOX Bet,1.80,2.05
7,2022-12-01 01:10:00+00:00,Oklahoma City Thunder,San Antonio Spurs,WynnBET,1.45,2.80
8,2022-12-01 02:10:00+00:00,Phoenix Suns,Chicago Bulls,WynnBET,2.90,1.43
9,2022-12-01 02:10:00+00:00,Denver Nuggets,Houston Rockets,WynnBET,1.11,5.50


In [16]:
# https://the-odds-api.com/sports-odds-data/sports-apis.html
# https://github.com/the-odds-api/samples-python/blob/master/sample-v4.py

In [17]:
import pymysql.cursors
from env import Env

env = Env()
connection = pymysql.connect(host=env.HOST_MYSQL, user=env.USER_MYSQL, password=env.PWD_MYSQL, database=env.DB_MYSQL)
cursor = connection.cursor()


query = "SELECT id, name  FROM Team"
df_teams = pd.read_sql(query, connection)
df_teams

/Users/benjamindrai/miniforge3/envs/nba_scrapping/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,name
0,atl,Atlanta Hawks
1,bkn,Brooklyn Nets
2,bos,Boston Celtics
3,cha,Charlotte Hornets
4,chi,Chicago Bulls
5,cle,Cleveland Cavaliers
6,dal,Dallas Mavericks
7,den,Denver Nuggets
8,det,Detroit Pistons
9,gs,Golden State Warriors


In [18]:
df_merge = df.merge(df_teams, left_on="home_team", right_on="name", how='left').drop(["name", "home_team"], axis=1).rename(columns={"id": "home_id"})
df_merge = df_merge.merge(df_teams, left_on="away_team", right_on="name", how='left').drop(["name", "away_team"], axis=1).rename(columns={"id": "away_id"})
df_merge

,commence_time,bookmaker,odd_home,odd_away,home_id,away_id
0,2022-12-01 00:10:00+00:00,WynnBET,1.44,2.63,orl,atl
1,2022-12-01 00:10:00+00:00,WynnBET,1.55,2.38,cle,phi
2,2022-12-01 00:40:00+00:00,WynnBET,1.25,4.00,bos,mia
3,2022-12-01 00:40:00+00:00,WynnBET,1.40,2.80,bkn,wsh
4,2022-12-01 00:40:00+00:00,WynnBET,1.44,2.63,ny,mil
5,2022-12-01 01:10:00+00:00,WynnBET,1.57,2.30,min,mem
6,2022-12-01 01:10:00+00:00,FOX Bet,1.80,2.05,no,tor
7,2022-12-01 01:10:00+00:00,WynnBET,1.45,2.80,okc,sa
8,2022-12-01 02:10:00+00:00,WynnBET,2.90,1.43,phx,chi
9,2022-12-01 02:10:00+00:00,WynnBET,1.11,5.50,den,hou


In [19]:
df_merge["date"] = df_merge["commence_time"].astype("datetime64[ns]")
df_merge.drop("commence_time", axis=1, inplace=True)
df_merge

/var/folders/jk/80s7smcd3hq_913l12tlcpnm0000gn/T/ipykernel_21359/961435053.py:1: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df_merge["date"] = df_merge["commence_time"].astype("datetime64[ns]")


,bookmaker,odd_home,odd_away,home_id,away_id,date
0,WynnBET,1.44,2.63,orl,atl,2022-12-01 00:10:00
1,WynnBET,1.55,2.38,cle,phi,2022-12-01 00:10:00
2,WynnBET,1.25,4.00,bos,mia,2022-12-01 00:40:00
3,WynnBET,1.40,2.80,bkn,wsh,2022-12-01 00:40:00
4,WynnBET,1.44,2.63,ny,mil,2022-12-01 00:40:00
5,WynnBET,1.57,2.30,min,mem,2022-12-01 01:10:00
6,FOX Bet,1.80,2.05,no,tor,2022-12-01 01:10:00
7,WynnBET,1.45,2.80,okc,sa,2022-12-01 01:10:00
8,WynnBET,2.90,1.43,phx,chi,2022-12-01 02:10:00
9,WynnBET,1.11,5.50,den,hou,2022-12-01 02:10:00


In [9]:
df_games = pd.read_sql("SELECT id, date, team_home_id, team_away_id FROM Game;", connection)
df_games["date"] = pd.to_datetime(df_games['date'])


/Users/benjamindrai/miniforge3/envs/nba_scrapping/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
import datetime

In [ ]:
df_games[df_games['date']== '2022-12-01']

In [ ]:
df_games.info()

In [ ]:
df_merge.merge(df_games, left_on=['date', 'home_id', 'away_id'], right_on=['date', 'team_home_id', 'team_away_id'], how='left')

In [ ]:
df_games.info()

In [ ]:
df_merge.merge(df_games, left_on=['date', 'home_id', 'away_id'], right_on=['date', 'team_home_id', 'team_away_id'], how='left')

In [14]:
connection.commit()
cursor.close()
connection.close()